In [ ]:
!pip install -U langgraph
!pip install langchain
!pip install langchainhub
!pip install langchain-openai
!pip install tiktoken

In [ ]:
import os

# # Set OPENAI API Key

os.environ["OPENAI_API_KEY"] = "your openai key"

# OR (load from .env file)

# from dotenv import load_dotenv
# load_dotenv("./.env")

# Basics of LangGraph

check out the langgraph docs here: 

- https://langchain-ai.github.io/langgraph/concepts/#background-agents-ai-workflows-as-graphs

LangGraph models agent workflows as state machines. You define the behavior of your agents using three key components:

- State: A shared data structure that represents the current snapshot of your application. It can be any Python type, but is typically a TypedDict or Pydantic BaseModel.

- Nodes: Python functions that encode the logic of your agents. They receive the current State as input, perform some computation or side-effect, and return an updated State.

- Edges: Control flow rules that determine which Node to execute next based on the current State. They can be conditional branches or fixed transitions.

By composing Nodes and Edges, you can create complex, looping workflows that evolve the State over time. The real power, though, comes from how LangGraph manages that State.

In [5]:
# This example was taken from the langchain documentation
# source: https://langchain-ai.github.io/langgraph/#overview
import json

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import END, MessageGraph
from langgraph.prebuilt.tool_node import ToolNode


# Define the function that determines whether to continue or not
def should_continue(messages):
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return END
    else:
        return "action"


# Define a new graph
workflow = MessageGraph()

tools = [TavilySearchResults(max_results=1)]
model = ChatOpenAI(model="gpt-4o").bind_tools(tools)

workflow.add_node("agent", model)
workflow.add_node("action", ToolNode(tools))

workflow.set_entry_point("agent")

# Conditional agent -> action OR agent -> END
workflow.add_conditional_edges(
    "agent",
    should_continue,
)

# Always transition `action` -> `agent`
workflow.add_edge("action", "agent")

memory = SqliteSaver.from_conn_string(":memory:") # Here we only save in-memory

# Setting the interrupt means that any time an action is called, the machine will stop
app = workflow.compile(checkpointer=memory, interrupt_before=["action"])

In [6]:
# Run the graph
thread = {"configurable": {"thread_id": "4"}}
for event in app.stream("what is the weather in sf currently", thread, stream_mode="values"):
    event[-1].pretty_print()

================================ Human Message =================================

what is the weather in sf currently
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_vYpZwWls8Lpl0pZ7rg2A5k14)
 Call ID: call_vYpZwWls8Lpl0pZ7rg2A5k14
  Args:
    query: current weather in San Francisco


The graph orchestrates everything:

The MessageGraph contains the agent's "Memory"
Conditional edges enable dynamic routing between the chatbot, tools, and the user.

Persistence makes it easy to stop, resume, and even rewind for full control over your application

With LangGraph, you can build complex, stateful agents by just defining your nodes, edges, and state schema - the graph take care of the rest.